In [1]:
import os, sys
import os.path as osp
root_dir = osp.dirname(osp.dirname(osp.dirname(os.getcwd())))
sys.path.append(osp.join(root_dir, 'src'))
from ml_modules.data import deepstabp_col
from ml_modules.data.retrievers import UniProt_Retriever

import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

retr = UniProt_Retriever()

In [7]:
prefix = 'cell'

filename = f'{prefix}.csv'
file_path = osp.join(deepstabp_col, filename)
accessions = np.loadtxt(
    file_path, usecols=0, skiprows=1, dtype=np.str_, delimiter=','
)

# file_path = '/Users/sebastian/Dropbox/projects/ai-thermostability/code/experiments/testing1/training_entries.txt'
# train_acc = np.loadtxt(file_path, dtype=np.str_, delimiter=',')
# file_path = '/Users/sebastian/Dropbox/projects/ai-thermostability/code/experiments/testing1/validation_entries.txt'
# valid_acc = np.loadtxt(file_path, dtype=np.str_, delimiter=',')

# accessions = np.concatenate(
#     ([a[:-5] for a in train_acc], [a[:-5] for a in valid_acc])
# )
print(accessions)
print(accessions.shape)

['A0A3B3IS91' 'A0AV96' 'A0AVF1' ... 'R4GMY8' 'R4GN35' 'R4GNH3']
(11568,)


In [8]:
successful_accessions, failed_accessions = retr.batch_retrieve(accessions)

R4GNH3      : 100%|##########| 11568/11568 [00:10<00:00, 1138.72it/s]


In [9]:
mol_weight_count = []
seq_len_count = []
organism_count = {}
interpro_count = {}
organism_tissue_count = {}
keyword_count = {}
reviewed_count = 0

pbar = tqdm(successful_accessions, ascii=True)
for accession in pbar:
    pbar.set_description(accession)

    with open(retr.path_to_data(accession), 'r') as f:
        content = f.readlines()

    all_acc = []
    isoform_acc = []
    tissues = []
    interpro = []
    keywords = []

    id_count = 0
    os_count = 0
    sq_count = 0

    for line in content:
        words = line.split()
        kw = words[0]

        if line.startswith('ID'):
            if id_count == 1:
                raise Exception('Multiple ID lines found')
            id_count += 1

            assert words[3].isdigit()
            seq_len = int(words[3])

            reviewed_count += 1 if words[2] == 'Reviewed;' else 0

        elif line.startswith('AC'):
            all_acc += [w[:-1] for w in words[1:]]

        elif line.startswith('OS'):
            if os_count == 1:
                continue
            os_count += 1

            organism = ' '.join(words[1:3])
        
        elif line.startswith('CC'):
            if words[1].startswith('IsoId'):
                isoform_acc.append(words[1][6:-1])

        elif line.startswith('DR'):
            # print(words)

            if words[1].startswith('InterPro'):
                interpro.append(words[2][:-1])
        
        elif line.startswith('KW'):
            if '{' in line[5:]:
                w = line[5:line.index('{')-1]
                keywords.append(w)
                continue
            elif '}' in line[5:] or '|' in line[5:]:
                continue

            for w in line[5:].split(';'):
                if w[-1] == '\n':
                    w = w[:-1]
                if w == '':
                    continue

                if w[-1] == '.':
                    w = w[:-1]
                if w[0] == ' ':
                    w = w[1:]
                
                keywords.append(w)
        
        elif line.startswith('RC'):
            if words[1].startswith('TISSUE'):
                tissues.append(words[1][7:-1])

        elif line.startswith('SQ'):
            if sq_count == 1:
                raise Exception('Multiple SQ lines found')
            sq_count += 1

            mol_weight = words[4]
            assert mol_weight.isdigit()
            mol_weight = int(mol_weight)

    all_acc = np.unique(all_acc)
    isoform_acc = np.unique(isoform_acc)
    tissues = np.unique(tissues)

    # tally all properties
    seq_len_count.append(seq_len)
    mol_weight_count.append(mol_weight)
    organism_count[organism] = organism_count.get(organism, 0) + 1
    for fam in interpro:
        interpro_count[fam] = interpro_count.get(fam, 0) + 1
    for tissue in tissues:
        tissue_count = organism_tissue_count.get(organism, {})
        organism_tissue_count[organism] = (
            tissue_count | {tissue: tissue_count.get(tissue, 0) + 1}
        )
    for keyword in keywords:
        keyword_count[keyword] = keyword_count.get(keyword, 0) + 1

  0%|          | 0/11528 [00:00<?, ?it/s]

In [10]:
fig, ax = plt.subplots()
bar_container = ax.bar(
    list(organism_count.keys()), list(organism_count.values())
)
ax.bar_label(bar_container, fmt='{:.0f}')
ax.tick_params(labelrotation=45)
ax.set_xlabel('organism')
ax.set_ylabel('count')
# plt.show()
plt.savefig(f'{prefix}-organisms.png', dpi=300, bbox_inches='tight')
plt.close()

keys = ['Membrane', 'Transmembrane', 'Ion channel']
fig, ax = plt.subplots()
bar_container = ax.bar(
    ['All'] + keys,
    [len(successful_accessions)] + [keyword_count[k] for k in keys]
)
ax.bar_label(bar_container, fmt='{:.0f}')
ax.set_xlabel('association')
ax.set_ylabel('count')
# plt.show()
plt.savefig(f'{prefix}-keywords.png', dpi=300, bbox_inches='tight')
plt.close()

plt.hist(seq_len_count, bins=100)
plt.xlabel('sequence length')
plt.ylabel('count')
plt.grid()
# plt.show()
plt.savefig(f'{prefix}-seq_len.png', dpi=300, bbox_inches='tight')
plt.close()

plt.hist(mol_weight_count, bins=100)
plt.xlabel('molecular weight')
plt.ylabel('count')
plt.grid()
# plt.show()
plt.savefig(f'{prefix}-mol_weight.png', dpi=300, bbox_inches='tight')
plt.close()

In [11]:
keyword_count

{'Direct protein sequencing': 2257,
 'Nucleus': 3421,
 'Reference proteome': 10804,
 'Secreted': 325,
 '3D-structure': 5787,
 'Alternative splicing': 5356,
 'Cytoplasm': 4175,
 'Methylation': 717,
 'mRNA processing': 332,
 'mRNA splicing': 261,
 'Repeat': 2247,
 'RNA-binding': 738,
 'Cell projection': 564,
 'Ciliopathy': 78,
 'Cilium': 117,
 'Disease variant': 2034,
 'Protein transport': 561,
 'TPR repeat': 126,
 'Transport': 1279,
 'Activator': 414,
 'Cell cycle': 560,
 'DNA-binding': 996,
 'Phosphoprotein': 5661,
 'Repressor': 432,
 'Transcription': 1306,
 'Transcription regulation': 1242,
 'Acetylation': 2954,
 'ATP-binding': 1413,
 'Ligase': 232,
 'Nucleotide-binding': 1768,
 'Ubl conjugation pathway': 501,
 'Calcium': 312,
 'Cell membrane': 1205,
 'Endocytosis': 100,
 'Endoplasmic reticulum': 743,
 'Lipid transport': 89,
 'Lipid-binding': 158,
 'Membrane': 3011,
 'Metal-binding': 2339,
 'Transmembrane': 1591,
 'Transmembrane helix': 1583,
 'Coiled coil': 1329,
 'Endosome': 404,
 '